In [2]:
import pickle 
import pandas as pd
import numpy as np 
from xgboost import XGBRFClassifier


In [3]:
# Loading data and making it ready
folder_name="EURUSD_H1_201801020000_202206210600_slatr_06"
name = "EURUSD_H1_201801020000_202206210600"
csv_path = "D:/dev/workspace/vscode/python/bos/data/csv/"
tp_to_sl = 1

train_path = csv_path + "preprocessed/" + folder_name + f"/train_{tp_to_sl:02}.csv"
test_path = csv_path + "preprocessed/" + folder_name + f"/test_{tp_to_sl:02}.csv"

train_df = pd.read_csv(train_path)

test_df = pd.read_csv(test_path)

test_buy = test_df.pop(test_df.Buy.name)
test_nothing= test_df.pop(test_df.do_nothing.name)
test_sell = test_df.pop(test_df.Sell.name)

train_buy = train_df.pop(train_df.Buy.name)
train_nothing = train_df.pop(train_df.do_nothing.name)
train_sell = train_df.pop(train_df.Sell.name)

train_date_time = train_df.pop(train_df.DateTime.name)
test_date_time = test_df.pop(test_df.DateTime.name)

train_close = train_df.pop("Close")
test_close = test_df.pop("Close")

train_open = train_df.Open[:]         #    .pop("Open")    
test_open = test_df.Open[:]             #.pop("Open")        
train_high = train_df.High[:]         #    .pop("High")    
test_high = test_df.High[:]             #.pop("High")        
train_low = train_df.Low[:]          #    .pop("Low")     
test_low = test_df.Low[:]              #.pop("Low")         

train_df_rc = []
test_df_rc = []

relative_columns= [
        "BOLLINGER_HBAND",
        "BOLLINGER_LBAND",
        "BOLLINGER_MAVG",
        "DONCHIAN_CHANNEL_HBAND",
        "DONCHIAN_CHANNEL_LBAND",
        "DONCHIAN_CHANNEL_MBAND",
        "EMA_INDICATOR_8",
        "EMA_INDICATOR_12",
        "EMA_INDICATOR_24",
        "EMA_INDICATOR_48",
        "EMA_INDICATOR_96",
        "EMA_INDICATOR_192",
        "SMA_INDICATOR_8",
        "SMA_INDICATOR_12",
        "SMA_INDICATOR_24",
        "SMA_INDICATOR_48",
        "SMA_INDICATOR_96",
        "SMA_INDICATOR_192",
        "Open",
        "High",
        "Low",
        "ICHIMOKU_A",
        "ICHIMOKU_B",
        "ICHIMOKU_BASE_LINE",
        "ICHIMOKU_CONVERSION_LINE",
        "KELTNER_CHANNEL_HBAND",
        "KELTNER_CHANNEL_LBAND",
        "KELTNER_CHANNEL_MBAND",
        "NEGATIVE_VOLUME_INDEX",
        "VOLUME_WEIGHTED_AVERAGE_PRICE"
    ]

for column in relative_columns:
    train_df_rc.append(train_df.pop(column))
    test_df_rc.append(test_df.pop(column))



In [4]:
train_data_name  = "EURUSD_H1_201801020000_202206210600"
train_tp_to_sl = 1
model_path = f"D:/dev/workspace/vscode/python/bos/model/models/XGBOOST/{train_data_name}_{train_tp_to_sl:02}_{10000}"

model = XGBRFClassifier()
model.load_model(f"{model_path}/model{train_tp_to_sl:02}.json")

In [5]:
with open(f"{model_path}/mean.pickle" , "rb") as f:
    mean = pickle.load(f)

with open(f"{model_path}/std.pickle" , "rb") as f:
    std = pickle.load(f)

X_train = (train_df - mean)/std
X_test = (test_df - mean)/std

# X_train = train_df
# X_test = test_df

y_train = []
for buy,nothing,sell in zip(train_buy,train_nothing,train_sell):
    if(buy == 1):
        y_train.append(0)
    elif(nothing == 1):
        y_train.append(1)
    elif(sell == 1):
        y_train.append(2)

y_test = []
for buy,nothing,sell in zip(test_buy,test_nothing,test_sell):
    if(buy == 1):
        y_test.append(0)
    elif(nothing == 1):
        y_test.append(1)
    elif(sell == 1):
        y_test.append(2)     
# print(X_train.shape,X_test.shape)


In [6]:
from sklearn.metrics import accuracy_score , confusion_matrix , classification_report
p_train = model.predict(X_train)
p_test = model.predict(X_test)

acc_train = accuracy_score(p_train,y_train)
acc_test = accuracy_score(p_test,y_test)
print(acc_train)
print(acc_test)

rep_train = classification_report(y_train,p_train)
rep_test = classification_report(y_test,p_test)
print(rep_train)
print(rep_test)


0.7145463390902457
0.5493006993006992
              precision    recall  f1-score   support

           0       0.94      0.35      0.51      5261
           1       0.68      0.98      0.80     13749
           2       0.81      0.41      0.55      5656

    accuracy                           0.71     24666
   macro avg       0.81      0.58      0.62     24666
weighted avg       0.76      0.71      0.68     24666

              precision    recall  f1-score   support

           0       0.31      0.15      0.21       557
           1       0.58      0.90      0.71      1614
           2       0.35      0.05      0.09       689

    accuracy                           0.55      2860
   macro avg       0.42      0.37      0.33      2860
weighted avg       0.48      0.55      0.46      2860



In [7]:
df = X_test.copy()
df = df * std + mean

df["DateTime"]= pd.to_datetime(test_date_time)
df=df.set_index(pd.DatetimeIndex(df["DateTime"]))

df["Signal"] = p_test
df["Close"] = test_close.tolist() 
df["High"] = test_high.tolist() 
df["Open"] = test_open.tolist() 
df["Low"] = test_low.tolist() 

df["Signal"].astype(np.float32)
df["Signal"] = df["Signal"]/10_000


In [9]:
# df[df.Signal == .0000].shape
# df[df.Signal == .0002].shape
# df.shape
# df.AVERAGE_TRUE_RANGE.describe()
# df.loc[df.Signal == 0].loc[df.EMA_INDICATOR_8_C < 100.0 ].describe()
60e-4

0.006

In [8]:
from backtesting import Strategy

def Signal():
    return df.Signal



class MyStrategy(Strategy):


    def init(self):
        super().init()
        self.c = 0
        self.initsize = 0.02
        self.mysize = self.initsize
        self.signal1 = self.I(Signal)

    def next(self):
        super().next()
        
        sl_to_atr = 6
        tp_to_atr = 6
        # if(self.signal1 != 1 and len(self.trades) == 0 and len(self.closed_trades) > 0 and self.closed_trades[-1].pl < 0):
        #     if self.mysize < .16:
        #         self.mysize = self.mysize * 2
        #         if self.mysize > .16:
        #             self.mysize = .16
        #     else:
        #         self.c += 1

        # elif len(self.closed_trades) > 0 and self.closed_trades[-1].pl > 0:
        #     if self.c > 0:
        #         self.c -= 1
        #     else:
        #         self.mysize = self.initsize
        
        if (    
            self.signal1 == 0 *1e-4 and
            # self.data.EMA_INDICATOR_48_C <= 1   and
            # self.data.EMA_INDICATOR_96_C <= 1   and
            # self.data.EMA_INDICATOR_8_C <= 1    and
            1):
            
            sl1 = self.data.Close[-1] - (sl_to_atr * min(self.data.AVERAGE_TRUE_RANGE[-1],0.0020)) #- self.data.Spread[-1]#*1e-4
            tp1 = self.data.Close[-1] + (tp_to_atr * max(self.data.AVERAGE_TRUE_RANGE[-1],0.0020)) #+ self.data.Spread[-1]#*1e-4
            self.buy(sl=sl1, tp=tp1, size=2000)

        if (
            self.signal1 == 2 *1e-4 and
            # self.data.EMA_INDICATOR_48_C >= 1   and
            # self.data.EMA_INDICATOR_96_C >= 1   and
            # self.data.EMA_INDICATOR_8_C >= 1    and
            1):
            
            sl1=self.data.Close[-1] + (sl_to_atr * min(self.data.AVERAGE_TRUE_RANGE[-1],0.0020)) #+ self.data.Spread[-1]#*1e-4
            tp1=self.data.Close[-1] - (tp_to_atr * max(self.data.AVERAGE_TRUE_RANGE[-1],0.0020)) #- self.data.Spread[-1]#*1e-4
            self.sell(sl=sl1, tp=tp1, size=2000)


from backtesting import Backtest

bt=Backtest(df, MyStrategy, cash=2_000, margin=1/100, commission=.002,hedging=True)
stat=bt.run()
# stat.to_csv(f"{path}/stat.csv")
stat

Start                     2022-01-03 00:00:00
End                       2022-06-17 05:00:00
Duration                    165 days 05:00:00
Exposure Time [%]                    64.86014
Equity Final [$]                   2248.49951
Equity Peak [$]                   3382.865805
Return [%]                          12.424976
Buy & Hold Return [%]                -7.44157
Return (Ann.) [%]                   27.882725
Volatility (Ann.) [%]              160.308919
Sharpe Ratio                         0.173931
Sortino Ratio                        0.477893
Calmar Ratio                         0.623322
Max. Drawdown [%]                  -44.732479
Avg. Drawdown [%]                   -4.924931
Max. Drawdown Duration       96 days 11:00:00
Avg. Drawdown Duration        4 days 14:00:00
# Trades                                  370
Win Rate [%]                        51.621622
Best Trade [%]                        1.35933
Worst Trade [%]                     -1.283689
Avg. Trade [%]                    

In [16]:
stat._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-2000,108,175,1.131264,1.137665,-12.803400,-0.005659,2022-01-07 12:00:00,2022-01-12 07:00:00,4 days 19:00:00
1,2000,82,183,1.129206,1.140960,23.508408,0.010409,2022-01-06 10:00:00,2022-01-12 15:00:00,6 days 05:00:00
2,2000,56,183,1.129576,1.141350,23.548260,0.010423,2022-01-05 08:00:00,2022-01-12 15:00:00,7 days 07:00:00
3,2000,55,183,1.129116,1.140870,23.508444,0.010410,2022-01-05 07:00:00,2022-01-12 15:00:00,7 days 08:00:00
4,2000,53,183,1.129426,1.141190,23.528320,0.010416,2022-01-05 05:00:00,2022-01-12 15:00:00,7 days 10:00:00
...,...,...,...,...,...,...,...,...,...,...
365,2000,2423,2748,1.069184,1.058352,-21.663994,-0.010131,2022-05-24 01:00:00,2022-06-10 14:00:00,17 days 13:00:00
366,2000,2422,2748,1.068994,1.057804,-22.379027,-0.010467,2022-05-24 00:00:00,2022-06-10 14:00:00,17 days 14:00:00
367,2000,2427,2749,1.066943,1.056880,-20.127267,-0.009432,2022-05-24 05:00:00,2022-06-10 15:00:00,17 days 10:00:00
368,2000,2426,2749,1.067203,1.056807,-20.793569,-0.009742,2022-05-24 04:00:00,2022-06-10 15:00:00,17 days 11:00:00


In [17]:
bt.plot()

Row(id='2475', ...)